In [1]:
import os
import numpy as np
import pandas as pd
import warnings

# Suppress scipy.signal UserWarnings about nperseg > data length
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    message=r"nperseg = .* is greater than input length"
)
# Suppress runtime warnings from empty‐slice, invalid‐value, etc.
warnings.filterwarnings("ignore", category=RuntimeWarning)

from code_descriptors_postural_control.stabilogram.stato import Stabilogram
from code_descriptors_postural_control.descriptors import compute_all_features

# ── 1) Paths ─────────────────────────────────────────────────────
data_folder = r"E:\USA_PD_2024\Sub_dataset_3\4.Tug_including_Demographics\PD"
output_file = r"E:\USA_PD_2024\Analysis\Anlysis_ppr3\COP_Features\COP_Data\SitToStand\L_R_COP_Data\cop_PD_features_SitToStand.csv"

# ── 2) Params ────────────────────────────────────────────────────
params_dic = {"sway_density_radius": 0.3}  # in centimeters

# ── 3) Gather files ──────────────────────────────────────────────
file_list = [f for f in os.listdir(data_folder) if f.lower().endswith(".csv")]
all_features = []

# ── 4) Main loop ─────────────────────────────────────────────────
for file_name in file_list:
    file_path = os.path.join(data_folder, file_name)
    try:
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()

        # — Filter to "SitToStand" only —
        sit = df[df["GeneralEvent"] == "SitToStand"]
        if sit.empty:
            print(f"[{file_name}] no 'SitToStand' rows → skip")
            continue

        # — Pull out COP arrays —
        Lx = sit["LCoP_X"].dropna().to_numpy()
        Ly = sit["LCoP_Y"].dropna().to_numpy()
        Rx = sit["RCoP_X"].dropna().to_numpy()
        Ry = sit["RCoP_Y"].dropna().to_numpy()

        # — Require at least 10 samples per axis —
        if any(arr.size < 10 for arr in (Lx, Ly, Rx, Ry)):
            print(f"[{file_name}] too few samples (<10) → skip")
            continue
        # — Ensure X/Y pairs align —
        if Lx.shape[0] != Ly.shape[0] or Rx.shape[0] != Ry.shape[0]:
            print(f"[{file_name}] X/Y length mismatch → skip")
            continue

        left_cop  = np.vstack((Lx, Ly)).T
        right_cop = np.vstack((Rx, Ry)).T

        # — Compute features with fail‐safe —
        try:
            st_l = Stabilogram();  st_l.from_array(left_cop, original_frequency=100)
            feat_l = compute_all_features(st_l, params_dic=params_dic)

            st_r = Stabilogram();  st_r.from_array(right_cop, original_frequency=100)
            feat_r = compute_all_features(st_r, params_dic=params_dic)
        except Exception as e_feat:
            print(f"[{file_name}] feature extraction error → skip ({e_feat})")
            continue

        # — Collate results —
        rec = {"File": file_name}
        rec.update({f"Left_{k}": v for k, v in feat_l.items()})
        rec.update({f"Right_{k}": v for k, v in feat_r.items()})
        all_features.append(rec)

    except Exception as e:
        print(f"[{file_name}] READ or PREP error → skip ({e})")

# ── 5) Save CSV ───────────────────────────────────────────────────
if all_features:
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    pd.DataFrame(all_features).to_csv(output_file, index=False)
    print(f"\n✅ Saved {len(all_features)} records to:\n   {output_file}")
else:
    print("\n⚠️  No valid data to save.")


[pd_nls002_tug.csv] feature extraction error → skip (zero-size array to reduction operation maximum which has no identity)
[pd_nls005_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls022_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls036_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls102_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls121_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls127_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls130_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls135_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls141_tug.csv] feature extraction error → skip (noverlap must be less than nperseg.)
[pd_nls142_tug.csv] feature extraction error → skip (noverlap must 